In [3]:
import pandas_datareader.data as web
import pandas as pd
import pyfolio as pf
from datetime import datetime
# q = web.get_data_alphavantage("AAPL", "av-daily", start=datetime(2017, 2, 9), end=datetime(2017, 5, 24), api_key="EL5PEVKNPNABAHJB")
# f = web.DataReader("usd/brl", "av-forex-daily", api_key="EL5PEVKNPNABAHJB")

c:\Users\wwwes\Documents\wwweslei-quantApp\.venv\lib\site-packages\pyfolio\pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [23]:
d = web.get_data_alphavantage("cme", api_key="EL5PEVKNPNABAHJB")
d


,open,high,low,close,volume
2002-12-06,39.00,43.20,38.900,42.90,5510200
2002-12-09,42.70,42.80,41.140,42.02,629900
2002-12-10,42.05,43.30,41.650,42.75,344600
2002-12-11,42.70,45.50,42.300,45.06,509800
2002-12-12,45.06,45.06,43.600,43.91,271100
...,...,...,...,...,...
2021-06-28,218.78,218.80,214.320,215.22,985892
2021-06-29,215.42,216.08,213.190,213.30,1243174
2021-06-30,213.19,213.71,211.220,212.68,1403001
2021-07-01,213.42,214.39,212.290,214.20,1222784


In [31]:
import quandl
key= "ALhffY5x9h5hoQNzYe_u"
quandl.get("MULTPL/SP500_REAL_PRICE_MONTH").tail(1)

,Value
Date,
2021-07-01,4319.94


In [36]:
import pandas_datareader.data as web
import datetime

start = datetime.datetime(2015, 1, 1)
end = datetime.datetime(2017, 1, 1)
web.get_data_fred('sp500')

,sp500
DATE,
2016-07-05,2088.55
2016-07-06,2099.73
2016-07-07,2097.90
2016-07-08,2129.90
2016-07-11,2137.16
...,...
2021-06-28,4290.61
2021-06-29,4291.80
2021-06-30,4297.50


In [104]:
import yfinance as yf
df = yf.download("ifix.sa")
df

[*********************100%***********************]  1 of 1 completed

1 Failed download:
- IFIX.SA: 1d data not available for startTime=-2208988800 and endTime=1625673486. Only 100 years worth of day granularity data are allowed to be fetched per request.


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [76]:
import profit
import json
from datetime import date

df = profit.ind()
df =df.groupby(pd.Grouper(freq='M')).last()[-12:]['Close']


In [98]:
data = {month: float(close/df.head(1)-1) for (month, close) in zip(df.index.strftime('%m'), df)}

for key in data:
    print(key, "   ", data[key])


08     0.0
09     -0.04796264428544117
10     -0.05451398323420786
11     0.09579446306192074
12     0.20063601324356695
01     0.16743652446940183
02     0.10733729835260486
03     0.17374633940162432
04     0.19648985095955473
05     0.27017480300697394
06     0.276072014410933
07     0.25946244238645866
